In [35]:
import bs4
import lxml
import pandas as pd
import urllib
from urllib import request

In [20]:
#Le 38 Riv
url = "https://www.38riv.com/billetterie-2/"
request_text = request.urlopen(url).read()
page = bs4.BeautifulSoup(request_text, "lxml")
link_list = page.find_all('a', {"class": "open_tickets"})
print(link_list)

[]


In [53]:
#L'international
url_2 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_2).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D = {key:[] for key in information}
for exhibition in exhibitions:
    info = exhibition.img.get('alt').split(' - ')
    D['date'].append(info[0])
    D['heure'].append(info[1])
    D['prix'].append(info[2])
n = len(D['date'])

p = []
for prix in D['prix']:
    p.append(prix[:-1])
D['prix'] = p

D['etablissement'] = ["L'international"]*n
D['nom'] = ['unknown']*n
D['adresse'] = ["5 rue Moret"]*n
D['arrondissement'] = [11]*n

data = pd.DataFrame(D)
data.tail()
#international_csv = data.to_csv('C:\\Users\\LENOVO\\Desktop\\data_international.csv', index =False, encoding = 'utf-8')



,nom,date,heure,prix,etablissement,adresse,arrondissement
25,unknown,Mardi 12 décembre,20h,8,L'international,5 rue Moret,11
26,unknown,Mercredi 13 décembre,20h,7,L'international,5 rue Moret,11
27,unknown,Jeudi 14 décembre,20h,8,L'international,5 rue Moret,11
28,unknown,Jeudi 14 décembre,00h,5,L'international,5 rue Moret,11
29,unknown,Vendredi 15 décembre,00h,7,L'international,5 rue Moret,11
